In [29]:
import requests

#샘플코드에서 기간을 변경해서 에러가 남
service_key='q+LmC5LyVljCscdUobqB7LjX4lt+PwuTwejcQGR+mgW0lJYVQPIe1v7kepGi8T1i/EaTH0gMyvi7WheDbzEXvA=='
url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
params ={'serviceKey' : service_key, 
         'pageNo' : '1', 'numOfRows' : '10', 'dataType' : 'XML', 
         'dataCd' : 'ASOS', 'dateCd' : 'HR', 'startDt' : '20120101', 
         'startHh' : '01', 'endDt' : '20211231', 'endHh' : '23', 'stnIds' : '108' }

response = requests.get(url, params=params)
#response.content
content=response.text #xml내용

In [30]:
response.content

b'<OpenAPI_ServiceResponse>\n\t<cmmMsgHeader>\n\t\t<errMsg>SERVICE ERROR</errMsg>\n\t\t<returnAuthMsg>HTTP_ERROR</returnAuthMsg>\n\t\t<returnReasonCode>04</returnReasonCode>\n\t</cmmMsgHeader>\n</OpenAPI_ServiceResponse>'

In [31]:
### xml을 DataFrame으로 변환하기 ###
#from os import name
import xml.etree.ElementTree as et
import pandas as pd
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

#bs4 사용하여 item 태그 분리
xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('item')

# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

#xml값 DataFrame으로 만들기
weather = pd.DataFrame(row_list, columns=name_list)
weather.head()

Empty DataFrame
Columns: []
Index: []

In [32]:
#DataFrame CSV 파일로 저장
weather.to_csv('weather_10.csv', encoding='utf-8')